## Install dependencies

In [38]:
#imports.... Run this each time after restarting the Kernel
#!pip install -e git+https://github.com/ibm-watson-data-lab/pixiedust@david-gateway-branch#egg=pixiedust
#!pip install --user --upgrade pixiedust
#!pip install watson_developer_cloud
import pixiedust
import json
import pandas
import datetime
import watson_developer_cloud as watson
import json
import time
import re
import pyspark
import sys
import pandas as pd
from botocore.client import Config
import ibm_boto3


#Notes from meeting with David Taieb
### Github Repo, Notebooks there and users create notebook from url.
### Add good markdown comments and documentation in Notebook
### Notebook 1, Fetch from COS(with audio files, we provide), run audio analysis, put transcript in their COS()
### notebook 2, TA, NLU, NLC => give data frame
### dashboard (UI), code is there and they can experiment and 
### publish it as webapp with pixie gateway, just show how to do it
### .



### Object storage client


In [39]:
# The code was removed by DSX for sharing.

## Speect to Text 

Create a service instance and add credentials below.

In [40]:
# The code was removed by DSX for sharing.

In [41]:
#STT

import json
import io
from os.path import join, dirname
from watson_developer_cloud import SpeechToTextV1

speech_to_text = SpeechToTextV1(
    username=username_STT,
    password=password_STT,
    x_watson_learning_opt_out=False
)


#get transcript Very basic one
def get_transcript(audio):
    transcript = json.dumps(speech_to_text.recognize(audio, content_type='audio/ogg', timestamps=True,
        word_confidence=True), indent=2)
    return transcript

def analyze_sample(sample):
    streaming_body = client.get_object(Bucket='watsoncallcenterthink18c8855d42de924dc38ccd02f3a8a50d7f', Key=sample)['Body']
    text = get_transcript(streaming_body.read())
    print(sample.split('.')[0] + '_text.json')
    print(text)
    client.put_object(Bucket='watsoncallcenterthink18c8855d42de924dc38ccd02f3a8a50d7f', Key = sample.split('.')[0] + '_text.json', Body = text)


## Analyze the selected file

In [42]:


    
file_list = ['sample1-addresschange-positive.ogg', 'sample2-address-negative.ogg', 'sample3-shirt-return-weather-chitchat.ogg', 'sample4-angryblender-sportschitchat-recovery.ogg', 'sample5-calibration-toneandcontext.ogg'] # we add audio files to COS pre-conference

analyze_sample(file_list[0])

sample1-addresschange-positive_text.json
{
  "results": [
    {
      "alternatives": [
        {
          "word_confidence": [
            [
              "good", 
              1.0
            ], 
            [
              "morning", 
              0.996
            ]
          ], 
          "confidence": 0.997, 
          "transcript": "good morning ", 
          "timestamps": [
            [
              "good", 
              1.78, 
              1.94
            ], 
            [
              "morning", 
              1.94, 
              2.37
            ]
          ]
        }
      ], 
      "final": true
    }, 
    {
      "alternatives": [
        {
          "word_confidence": [
            [
              "can", 
              1.0
            ], 
            [
              "you", 
              1.0
            ], 
            [
              "give", 
              1.0
            ], 
            [
              "me", 
              1.0
            ], 
            [
